In [ ]:
# numpy, pandas 가져오기


### 보스턴 주택 가격 데이터 세트를 Perceptron 기반에서 학습 및 테스트하기 위한 데이터 로드
* 사이킷런에서 보스턴 주택 가격 데이터 세트를 로드하고 이를 DataFrame으로 생성

In [ ]:
# sklearn 버전 확인
import sklearn
print(sklearn.__version__)

1.0.2


In [ ]:
# 이전 버전 설치
!pip install scikit-learn==1.0.2

In [ ]:
# 데이터 불러오기
from sklearn.datasets import load_boston
boston = load_boston()

print(boston.feature_names)

In [ ]:
import numpy as np
import pandas as pd

# DataFrame 생성
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)

# target 열 추가('PRICE')
bostonDF['PRICE'] = boston.target

bostonDF.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [ ]:
len(bostonDF)

506

### 데이터 스케일링
* 사이킷런의 MinMaxScaler를 이용하여 개별 feature값은 0~1사이 값으로 변환후 학습 적용.

In [ ]:
# 'RM', 'LSTAT' 속성에만 MinMaxScaling 적용
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(bostonDF[['RM', 'LSTAT']])
scaled_features

array([[0.57750527, 0.08967991],
       [0.5479977 , 0.2044702 ],
       [0.6943859 , 0.06346578],
       ...,
       [0.65433991, 0.10789183],
       [0.61946733, 0.13107064],
       [0.47307913, 0.16970199]])

### **get_update_value**: Weight와 Bias의 Update 값(편미분값)을 계산하는 함수
* w0은 Bias
* w1은 RM(방의 개수) 피처의 Weight 값
* w2는 LSTAT(하위계층 비율) 피처의 Weight 값
* N은 입력 데이터 건수


In [ ]:
# gradient_descent()함수에서 반복적으로 호출되면서 update될 weight/bias 값을 계산하는 함수
# rm은 RM(방 개수), lstat(하위계층 비율), target은 PRICE임. 전체 array가 다 입력됨
# 반환 값은 weight와 bias가 update되어야 할 값과 MSE(Mean Squared Error) 값을 loss로 반환
def get_update_value(bias, w1, w2, rm, lstat, target, learning_rate=0.01):

    # 데이터 건수
    N = len(target)

    # 예측 값 (rm, lstat이 배열, 따라서 506개의 값이 들어온다.)
    predicted = (w1*rm) + (w2*lstat) + bias

    # 실제값과 예측값의 차이 (배열 뺄셈)
    diff = target - predicted

    # bias 를 array 기반으로 구하기 위해서 설정
    bias_array = np.ones((N, ))

    # weight와 bias를 얼마나 update할 것인지를 계산
    # (-2/n) * 학습률 * (행렬의 곱)
    # 행렬의 곱 -> 내적, rm을 전치행렬로 만들기
    w1_update = (-2/N) * learning_rate * (np.dot(rm.T, diff))
    w2_update = (-2/N) * learning_rate * (np.dot(lstat.T, diff))

    # diff만 필요하니까 1로 찬 배열로 곱해주자.
    bias_update = (-2/N) * learning_rate * (np.dot(bias_array.T, diff))

    # Mean Squared Error값을 계산.
    mse_loss = np.mean(np.square(diff))

    # weight와 bias가 update되어야 할 값과 Mean Squared Error 값을 반환
    return bias_update, w1_update, w2_update, mse_loss

### **gradient_descent** : 경사하강법을 적용(update)하는 함수
* iter_epochs 수만큼 반복적으로 get_update_value()를 호출하여 update될 weight/bias값을 구한 뒤 Weight/Bias를 Update적용.
*  verbose: 함수 수행시 발생하는 상세한 정보들을 표준 출력으로 자세히 내보낼 것인가 (보통 0 은 출력하지 않고, 1은 자세히, 2는 함축적인 정보만 출력)

In [ ]:
# RM, LSTAT feature array와 PRICE target array를 입력 받아서 iter_epochs수만큼 반복적으로 Weight와 Bias를 update적용.
def gradient_descent(features, target, iter_epochs=1000, verbose=1):

    # w1, w2는 numpy array 연산을 위해 1차원 array로 변환하되 초기 값은 0으로 설정
    # bias도 1차원 array로 변환하되 초기 값은 1로 설정.
    w1 = np.zeros((1,))
    w2 = np.zeros((1,))
    bias = np.ones((1, ))
    print('최초 w1, w2, bias:', w1, w2, bias)

    # learning_rate와 RM, LSTAT 피처 지정. 호출 시 numpy array형태로 RM과 LSTAT으로 된 2차원 feature가 입력됨.
    learning_rate = 0.01
    # 행은 다, 열은 0번째만 / features = scaled_features
    rm = features[:, 0]
    lstat = features[:, 1]


    # iter_epochs 수만큼 반복하면서 weight와 bias update 수행.
    for i in range(iter_epochs):

        # weight/bias update 값 계산
        bias_update, w1_update, w2_update, loss = get_update_value(bias, w1, w2, rm, lstat, target, learning_rate=0.01)

        # weight/bias의 update 적용.
        w1 = w1 - w1_update
        w2 = w2 - w2_update
        bias = bias - bias_update

        if verbose:
            print('Epoch:', i+1,'/', iter_epochs)
            print('w1:', w1, 'w2:', w2, 'bias:', bias, 'loss:', loss)

    return w1, w2, bias

### 경사하강법(Gradient Descent) 적용

In [ ]:
# `  함수 호출로 학습(epochs 5000번)
w1, w2, bias = gradient_descent(scaled_features, bostonDF['PRICE'].values, 5000)
print('####### 최종 w1, w2, bias #######')
print(w1, w2, bias)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
Epoch: 2502 / 5000
w1: [23.64774013] w2: [-21.39182946] bias: [16.64483221] loss: 30.988398959658827
Epoch: 2503 / 5000
w1: [23.64936191] w2: [-21.39428063] bias: [16.64472504] loss: 30.98753274351138
Epoch: 2504 / 5000
w1: [23.6509823] w2: [-21.39672937] bias: [16.64461787] loss: 30.986668167355337
Epoch: 2505 / 5000
w1: [23.6526013] w2: [-21.39917568] bias: [16.64451069] loss: 30.98580522806791
Epoch: 2506 / 5000
w1: [23.65421892] w2: [-21.40161958] bias: [16.64440351] loss: 30.984943922532253
Epoch: 2507 / 5000
w1: [23.65583516] w2: [-21.40406106] bias: [16.64429631] loss: 30.984084247637455
Epoch: 2508 / 5000
w1: [23.65745002] w2: [-21.40650013] bias: [16.64418911] loss: 30.98322620027854
Epoch: 2509 / 5000
w1: [23.65906349] w2: [-21.40893678] bias: [16.6440819] loss: 30.982369777356457
Epoch: 2510 / 5000
w1: [23.66067558] w2: [-21.41137102] bias: [16.64397468] loss: 30.98151497577804
Epoch: 2511 / 5000
w1: [23.6622863] w2: [-21.41380285] bias: [

### 도출된 Weight와 Bias를 이용하여 Price 예측
* 예측 feature 역시 0~1사이의 scaled값을 이용하고 Weight와 bias를 적용하여 예측값 계산.

In [ ]:
# weight와 bias로 예측 값 생성 후 dataFrame에 'PREDICTED_PRICE' 컬럼 추가
predicted = scaled_features[:, 0]* w1 + scaled_features[:, 1]*w2 + bias
bostonDF['PREDICTED_PRICE'] = predicted
bostonDF.head()


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE,PREDICTED_PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0,28.948584
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6,25.489438
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7,32.538194
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4,32.337266
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2,31.506524
